## Existing Agent Executor

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import create_openai_functions_agent
from langchain_core.prompts import PromptTemplate
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from permchain.langgraph import Actor, DecisionPoint, Graph

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

from langchain_core.agents import AgentFinish
# Define decision-making logic
def should_continue(data):
    # Logic to decide whether to continue in the loop or exit
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    else:
        return "continue"
    
def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    observation = {t.name: t for t in tools}[agent_action.tool].invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data
    
    

# Define agents
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)

# Define a new graph
workflow = Graph()
decision_point = DecisionPoint("exit?", should_continue)
llm_agent = Actor("agent", agent)
tool_actor = Actor("tools", RunnableLambda(execute_tools))

# Register actors
workflow.register(llm_agent)
workflow.register(tool_actor)
workflow.register(decision_point)

# Define connections with conditional logic
workflow.connect(llm_agent, decision_point)
workflow.branch(decision_point, tool_actor, condition="continue")
workflow.branch(decision_point, None, condition="exit")  # Exit the workflow
workflow.connect(tool_actor, llm_agent)

# Define entry point and execute the graph
workflow.set_entry_point(llm_agent)
chain = workflow.compile()

In [2]:
chain.invoke({"input": "what is the weather in sf", "intermediate_steps": []})

{'input': 'what is the weather in sf',
 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query":"weather in San Francisco"}'}})]),
   [{'url': 'https://www.weather2travel.com/california/san-francisco/january/',
     'content': 'San Francisco weather in January 2024 Expect 13°C daytime maximum temperatures  long-term weather averages for San Francisco in January before you book your next holiday to California in 2024/2025.  San Francisco January sunrise & sunset times  How sunny is it in San Francisco in January?San Francisco weather in January 2024 Expect 13°C daytime maximum temperatures in the shade with on average 6 hours of sunshine per day in San Francisco in January. Check more 

## Reflexion Agent

In [14]:
from langchain.agents import AgentExecutor, BaseMultiActionAgent, Tool
from langchain.schema import AgentAction, AgentFinish
from langchain_core.language_models.chat_models import BaseChatModel
from langchain.chains import LLMChain

from langchain.globals import set_llm_cache

from dotenv import load_dotenv

from pydantic import BaseModel

from langchain.chat_models import ChatOpenAI
from langchain.cache import SQLiteCache

from langchain_core.output_parsers import BaseOutputParser

from langchain.prompts.chat import ChatPromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.pydantic_v1 import BaseModel
import os

from langchain.agents import AgentType, initialize_agent, load_tools

set_llm_cache(SQLiteCache(database_path=".langchain.db"))


llm = ChatOpenAI(
    temperature=0.0,
    max_tokens=2000,
    max_retries=100,
    model="gpt-4-1106-preview",
)

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)

NEXT_STEP_TEMPLATE = """You are expert researcher trying answer a question ~250 words. You are asked to answer the following question: {question}

The way you are going to answer the question is as follows:

1. Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        _ You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.
2. Reflect and critique your answer. Specifically, you should:
    - Think about what is missing from your answer.
    - Think about what is superfluous in your answer.
    - Think about what search query you should use next to improve your answer.
  Give your answer in exactly 2 parts. The first should address what is missing from your answer. The second should address what could be removed from your answer. Your should be VERY harsh as we really want to improve the answer.
3. Give the search query you came up with to improve your answer.

Previous steps: 

{previous_steps}

===

Format your answer as follows:

Revised answer: [give your revised answer based on the previous critique and new information from the search engine then the "References" section]
Critique: [give your harsh critique of your revised answer in 2 parts: what is missing and what is superfluous]
Search query: [give the new search query you came up with to enter into the search engine to improve your answer. If you have more than one, make sure they are comma separated and in quotes]

SAY NOTHING else please."""

INITIAL_ANSWER_TEMPLATE = """You are expert researcher trying answer a question ~250 words. You are asked to answer the following question: {question}

The way you are going to answer the question is as follows:

1. Give a detailed in ~250 words.
2. Reflect and critique your answer. Specifically, you should:
    - Think about what is missing from your answer.
    - Think about what is superfluous in your answer.
    - Think about what search query you should use next to improve your answer.
  Give your answer in exactly 2 parts. The first should address what is missing from your answer. The second should address what could be removed from your answer. Your should be VERY harsh as we really want to improve the answer.
3. Give the search query you came up with to improve your answer.

===

Format your answer as follows:

Answer: [give your initial answer]
Critique: [give your harsh critique of your answer in 2 parts: what is missing and what is superfluous]
Search query: [give the search query you came up with to improve your answer. If you have more than one, make sure they are comma separated and in quotes]

SAY NOTHING else please."""


class ReflexionStep(BaseModel):
    """A single step in the reflexion process."""

    answer: str
    critique: str
    search_query: str

    def __str__(self):
        return f"Answer: {self.answer}\nCritique: {self.critique}\nSearch query: {self.search_query}"

def _parse_reflexion_step(output: str) -> tuple[str, str, str]:
    # find answer using .split()
    if ("Answer:" not in output and "Revised answer:" not in output) or not "Critique:" in output or not "Search query:" in output:
        raise ValueError(f"The output is not formatted correctly. Output: {output}")
    if "Answer:" in output:
        answer = output.split("Answer:")[1].split("Critique:")[0].strip()
    else:
        answer = output.split("Revised answer:")[1].split("Critique:")[0].strip()
    critique = output.split("Critique:")[1].split("Search query:")[0].strip()
    search_query = output.split("Search query:")[1].strip()
    return answer, critique, search_query

class ReflexionStepParser(BaseOutputParser[ReflexionStep]):
    """Parser for the reflexion step."""

    def parse(self, output: str) -> ReflexionStep:
        """Parse the output."""
        # try to find answer or initial answer
        answer, critique, search_query = _parse_reflexion_step(output)
        return ReflexionStep(
            answer=answer, critique=critique, search_query=search_query
        )

In [21]:
initial_chain = RunnablePassthrough.assign(
    agent_outcome = ChatPromptTemplate.from_template(INITIAL_ANSWER_TEMPLATE) | llm | ReflexionStepParser() | (lambda x: AgentAction(
                    tool="tavily_search_results_json",
                    tool_input=x.search_query,
                    log=str(x),
                ))
)

def prep_next(inputs):
    intermediate_steps = inputs["intermediate_steps"]
    previous_steps = list[str]()

    for i, (action, observation) in enumerate(intermediate_steps, start=1):
        last_step_str = f"""Step {i}:

{action.log}

Search output for "{action.tool_input}":

{observation}"""
        previous_steps.append(last_step_str)

    previous_steps_str = "\n\n".join(previous_steps)
    inputs["previous_steps"] = previous_steps_str
    return inputs
    
next_chain = RunnablePassthrough.assign(
    agent_outcome = prep_next | ChatPromptTemplate.from_template(NEXT_STEP_TEMPLATE) | llm | ReflexionStepParser() | (lambda x: AgentAction(
                tool="tavily_search_results_json",
                tool_input=x.search_query,
                log=str(x),
            ))
)

def finish(inputs):
    intermediate_steps = inputs["intermediate_steps"]
    last_action, _ = intermediate_steps[-1]
    last_step_str = last_action.log
    # extract answer
    answer, _, _ = _parse_reflexion_step(last_step_str)

    first_action, _ = intermediate_steps[0]
    first_step_str = first_action.log
    # extract answer
    initial_answer, _, _ = _parse_reflexion_step(first_step_str)

    return AgentFinish(
        log="Reached max steps.",
        return_values={"output": answer, "initial_answer": initial_answer},
    )


def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    observation = {t.name: t for t in tools}[agent_action.tool].invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data


In [22]:
workflow = Graph()
decision_point = DecisionPoint("exit?", lambda x: "exit" if len(x['intermediate_steps']) >= 2 else "continue")
initial_answer_actor = Actor("initial", initial_chain)
next_step_actor = Actor("next", next_chain)
finish_actor = Actor("finish", RunnableLambda(finish))
tool_actor = Actor("tools", RunnableLambda(execute_tools))

# Register actors
workflow.register(initial_answer_actor)
workflow.register(next_step_actor)
workflow.register(finish_actor)
workflow.register(decision_point)
workflow.register(tool_actor)

# Define connections with conditional logic
workflow.connect(initial_answer_actor, tool_actor)
workflow.branch(decision_point, next_step_actor, condition="continue")
workflow.connect(next_step_actor, tool_actor)
workflow.branch(decision_point, finish_actor, condition="exit")  # Exit the workflow
workflow.connect(tool_actor, decision_point)

# Define entry point and execute the graph
workflow.set_entry_point(initial_answer_actor)
workflow.set_finish_point(finish_actor)
chain = workflow.compile()

chain.invoke({"question": "what is the weather in sf", "intermediate_steps": []})

AgentFinish(return_values={'output': "The current weather in San Francisco is characterized by a daytime maximum temperature of 13°C in January, with an average of 6 hours of sunshine per day and 10 days of some rainfall. Nighttime temperatures typically drop to around 7°C. There is no significant heat and humidity, and the monthly rainfall averages at 125 mm. The UV index is low at 2, and the sea temperature averages at 11°C. These conditions reflect San Francisco's Mediterranean-like climate, with its wet winters and dry summers. However, weather can vary between neighborhoods due to microclimates, so localized weather advisories should be checked. For real-time updates and specific forecasts, including any weather advisories, it is recommended to consult a reliable weather service[1].\n\nReferences:\n- [1] https://www.weather2travel.com/california/san-francisco/january/", 'initial_answer': "The weather in San Francisco (SF) is characterized by a mild, Mediterranean-like climate with